In [ ]:
!pip install wandb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.3/277.3 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00


In [ ]:
import wandb
wandb.login()


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
pip install tensorflow numpy

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

def build_lstm_model(input_dim, output_dim, embedding_dim, lstm_units):
    """
    Construiește un model LSTM simplu.

    Args:
    input_dim (int): Dimensiunea vocabularului (numărul de cuvinte unice).
    output_dim (int): Dimensiunea vectorului de ieșire pentru fiecare token.
    embedding_dim (int): Dimensiunea spațiului de încorporare.
    lstm_units (int): Numărul de unități în stratul LSTM.

    Returns:
    tf.keras.Model: Modelul LSTM compilat.
    """
    model = Sequential([
        Embedding(input_dim=input_dim, output_dim=embedding_dim, input_length=None),
        LSTM(units=lstm_units, return_sequences=True),
        Dense(output_dim, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


The actual code


In [ ]:
# THE CLASS FOR EMBEDDED WORDS WITH LABELS
import numpy as np

class LSTM:
    def __init__(self, input_dim, hidden_dim, output_dim, num_classes):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_classes = num_classes  # Number of classes possible for labeling
        self.init_weights()

    def init_weights(self):
        # Weight initialization for LSTM gates
        self.Wf = np.random.randn(self.hidden_dim, self.input_dim + self.hidden_dim)
        self.Wi = np.random.randn(self.hidden_dim, self.input_dim + self.hidden_dim)
        self.Wc = np.random.randn(self.hidden_dim, self.input_dim + self.hidden_dim)
        self.Wo = np.random.randn(self.hidden_dim, self.input_dim + self.hidden_dim)
        self.Wy = np.random.randn(self.num_classes, self.hidden_dim)  # Output layer weights

        # Bias initialization
        self.bf = np.zeros((self.hidden_dim, 1))
        self.bi = np.zeros((self.hidden_dim, 1))
        self.bc = np.zeros((self.hidden_dim, 1))
        self.bo = np.zeros((self.hidden_dim, 1))
        self.by = np.zeros((self.num_classes, 1))

    # Activation functions and their derivatives
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def tanh(self, x):
        return np.tanh(x)

    def sigmoid_derivative(self, sigmoid_output):
        return sigmoid_output * (1 - sigmoid_output)

    def tanh_derivative(self, tanh_output):
        return 1 - np.power(tanh_output, 2)

    def softmax(self, z):
        e_z = np.exp(z - np.max(z))
        return e_z / e_z.sum(axis=0)

    def cross_entropy_loss(outputs, targets):
        """
        Computes the cross-entropy loss between targets and predictions.

        Args:
        outputs: Numpy array of model outputs; these should be probabilities obtained
                via softmax and should have shape (batch_size, sequence_length, num_classes)
        targets: Numpy array of target labels in one-hot encoded form;
                should have shape (batch_size, sequence_length, num_classes)

        Returns:
        float: The mean cross-entropy loss over all samples and timesteps.
        """
        # Ensure numerical stability with clipping
        epsilon = 1e-12
        outputs = np.clip(outputs, epsilon, 1. - epsilon)

        # Compute cross-entropy
        ce_loss = -np.sum(targets * np.log(outputs)) / outputs.shape[0]

        return ce_loss

    def forward(self, x):
        """
        Performs the forward pass of the LSTM.

        Args:
        x: Input data, shape (batch_size, sequence_length, input_dim)

        Returns:
        A tuple (all_hiddens, all_outputs) where:
        - all_hiddens is an array of hidden states for the whole sequence
        - all_outputs is the softmax output for each word in the sequence
        """
        batch_size, sequence_length, _ = x.shape
        h_prev = np.zeros((batch_size, self.hidden_dim))
        c_prev = np.zeros((batch_size, self.hidden_dim))
        all_hiddens = np.zeros((batch_size, sequence_length, self.hidden_dim))
        all_outputs = np.zeros((batch_size, sequence_length, self.num_classes))

        for t in range(sequence_length):
            xt = x[:, t, :]
            combined = np.hstack((h_prev, xt))  # Combine the hidden state with the input vector

            # LSTM gate computations
            ft = self.sigmoid(np.dot(combined, self.Wf.T) + self.bf.T)  # Forget gate
            it = self.sigmoid(np.dot(combined, self.Wi.T) + self.bi.T)  # Input gate
            ct_ = self.tanh(np.dot(combined, self.Wc.T) + self.bc.T)     # Cell candidate
            ot = self.sigmoid(np.dot(combined, self.Wo.T) + self.bo.T)  # Output gate

            # Update the cell and hidden state
            c_curr = ft * c_prev + it * ct_
            h_curr = ot * self.tanh(c_curr)

            # Store the current hidden state
            h_prev = h_curr
            c_prev = c_curr
            all_hiddens[:, t, :] = h_curr

            # Compute the output layer
            output_raw = np.dot(h_curr, self.Wy.T) + self.by.T
            all_outputs[:, t, :] = self.softmax(output_raw)

        return all_hiddens, all_outputs

    def backward(self, x, h, c, y, targets, dh_next, dc_next):
        """
        Performs the backward pass of the LSTM.

        Args:
        x: Input data at all time steps (batch_size, sequence_length, input_dim)
        h: Hidden states at all time steps (batch_size, sequence_length, hidden_dim)
        c: Cell states at all time steps (batch_size, sequence_length, hidden_dim)
        y: Outputs at all time steps (batch_size, sequence_length, output_dim)
        targets: Ground truth for the outputs
        dh_next: Gradient of the loss w.r.t. the next hidden state
        dc_next: Gradient of the loss w.r.t. the next cell state
        """
        batch_size, sequence_length, _ = x.shape

        # Gradient accumulators for LSTM parameters
        dWf, dWi, dWc, dWo = [np.zeros_like(w) for w in (self.Wf, self.Wi, self.Wc, self.Wo)]
        dbf, dbi, dbc, dbo = [np.zeros_like(b) for b in (self.bf, self.bi, self.bc, self.bo)]
        dWy = np.zeros_like(self.Wy)
        dby = np.zeros_like(self.by)

        for t in reversed(range(sequence_length)):
            # Gradient of the output layer
            dy = y[:, t, :] - targets[:, t, :]
            dWy += np.dot(dy.T, h[:, t, :])
            dby += np.sum(dy, axis=0, keepdims=True).T

            # Total gradients for backpropagation
            dh = np.dot(dy, self.Wy) + dh_next

            # Output gate gradients
            do = dh * np.tanh(c[:, t, :])
            do = self.sigmoid_derivative(self.sigmoid(np.dot(h[:, t-1, :], self.Wo) + self.bo)) * do

            # Cell state gradients
            dc = (dc_next + dh * self.sigmoid(np.dot(h[:, t-1, :], self.Wo) + self.bo) * self.tanh_derivative(np.tanh(c[:, t, :])))
            dc += do * self.tanh_derivative(np.tanh(c[:, t, :]))

            # Input and forget gate gradients
            di = dc * np.tanh(np.dot(h[:, t-1, :], self.Wc) + self.bc)
            di = self.sigmoid_derivative(self.sigmoid(np.dot(h[:, t-1, :], self.Wi) + self.bi)) * di
            df = dc * c[:, t-1, :]
            df = self.sigmoid_derivative(self.sigmoid(np.dot(h[:, t-1, :], self.Wf) + self.bf)) * df

            # Cell candidate gradients
            dc_ = dc * self.sigmoid(np.dot(h[:, t-1, :], self.Wi) + self.bi)
            dc_ = self.tanh_derivative(np.tanh(np.dot(h[:, t-1, :], self.Wc) + self.bc)) * dc_

            # Accumulate gradients
            dWf += np.dot(df.T, np.hstack([h[:, t-1, :], x[:, t, :]]))
            dWi += np.dot(di.T, np.hstack([h[:, t-1, :], x[:, t, :]]))
            dWc += np.dot(dc_.T, np.hstack([h[:, t-1, :], x[:, t, :]]))
            dWo += np.dot(do.T, np.hstack([h[:, t-1, :], x[:, t, :]]))

            dbf += np.sum(df, axis=0)
            dbi += np.sum(di, axis=0)
            dbc += np.sum(dc_, axis=0)
            dbo += np.sum(do, axis=0)

            # Prepare for the next step
            dh_next = np.dot(df, self.Wf) + np.dot(di, self.Wi) + np.dot(dc_, self.Wc) + np.dot(do, self.Wo)
            dc_next = df * c[:, t-1, :] + di * dc_ + do * np.tanh(c[:, t, :])

        # Update parameters
        self.update_weights(dWf, dWi, dWc, dWo, dbf, dbi, dbc, dbo, dWy, dby)

    def update_weights(self, dWf, dWi, dWc, dWo, dbf, dbi, dbc, dbo, dWy, dby):
        # Apply learning rate and update weights and biases
        learning_rate = self.learning_rate
        self.Wf -= learning_rate * dWf
        self.Wi -= learning_rate * dWi
        self.Wc -= learning_rate * dWc
        self.Wo -= learning_rate * dWo
        self.bf -= learning_rate * dbf
        self.bi -= learning_rate * dbi
        self.bc -= learning_rate * dbc
        self.bo -= learning_rate * dbo
        self.Wy -= learning_rate * dWy
        self.by -= learning_rate * dby

    def train(self, train_data, train_targets, epochs, batch_size=1, save_path=None, validation_data=None):
        for epoch in range(epochs):
            epoch_losses = []
            for i in range(0, len(train_data), batch_size):
                x = train_data[i:i+batch_size]
                y = train_targets[i:i+batch_size]

                # Assuming 'forward' returns softmax probabilities
                _, outputs = self.forward(x)

                # Calculate cross-entropy loss
                loss = self.cross_entropy_loss(outputs, y)
                epoch_losses.append(loss)

                # Backward pass and update
                gradients = self.backward(x, outputs, y)
                self.update_weights(gradients)

            # Average loss for the epoch
            average_loss = np.mean(epoch_losses)
            print(f'Epoch {epoch + 1}/{epochs}, Loss: {average_loss}')

            if save_path:
                self.save_model(f'{save_path}_epoch_{epoch + 1}.json')

            if validation_data:
                val_loss = self.evaluate(validation_data[0], validation_data[1])
                print(f'Validation Loss: {val_loss}')

    def evaluate(self, val_data, val_targets):
        """
        Evaluate the model on validation data.

        Args:
        val_data: Numpy array of validation data.
        val_targets: Numpy array of validation targets.

        Returns:
        Float, the loss computed on the validation data.
        """
        total_loss = 0
        for i in range(len(val_data)):
            x = val_data[i:i+1]
            y = val_targets[i:i+1]
            _, outputs = self.forward(x)
            loss = np.mean((outputs - y) ** 2)
            total_loss += loss
        return total_loss / len(val_data)

    def decode_predictions(probabilities, classes):
        """
        Convert probabilities to class labels.

        Args:
        probabilities: np.array, shape (batch_size, sequence_length, num_classes)
        classes: list, the names of the classes corresponding to the indices in the last dimension of probabilities

        Returns:
        List of lists, where each sublist contains the predicted class labels for a sequence.
        """
        # Get the index of the max probability for each timestep
        class_indices = np.argmax(probabilities, axis=-1)

        # Convert indices to class labels
        decoded_labels = [[classes[idx] for idx in sequence] for sequence in class_indices]
        return decoded_labels

    def save_model(self, filename):
        """ Saves the model's parameters to a JSON file. """
        model_params = {
            'Wf': self.Wf.tolist(), 'Wi': self.Wi.tolist(),
            'Wc': self.Wc.tolist(), 'Wo': self.Wo.tolist(),
            'Wy': self.Wy.tolist(),
            'bf': self.bf.tolist(), 'bi': self.bi.tolist(),
            'bc': self.bc.tolist(), 'bo': self.bo.tolist(),
            'by': self.by.tolist()
        }
        with open(filename, 'w') as f:
            json.dump(model_params, f)

    def load_model(self, filename):
        """ Loads model parameters from a JSON file. """
        with open(filename, 'r') as f:
            model_params = json.load(f)
        self.Wf = np.array(model_params['Wf'])
        self.Wi = np.array(model_params['Wi'])
        self.Wc = np.array(model_params['Wc'])
        self.Wo = np.array(model_params['Wo'])
        self.Wy = np.array(model_params['Wy'])
        self.bf = np.array(model_params['bf'])
        self.bi = np.array(model_params['bi'])
        self.bc = np.array(model_params['bc'])
        self.bo = np.array(model_params['bo'])
        self.by = np.array(model_params['by'])


Code that helped me build the actual code


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import math

data = pd.read_csv("")
data = data.ffill()

In [ ]:
PREDICTORS = ["","",""] # columns to use for prediction
TARGET = "" # the predicted value

scaler = StandardScaler()
data[PREDICTORS] = scaler.fit_transform(data[PREDICTORS])

np.random.seed(0)
split_data = np.split(data, [int(.7*len(data)), int(.85*len(data))])
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = [[d[PREDICTORS].to_numpy(), d[[TARGET]].to_numpy()] for d in split_data]

In [ ]:
def init_params(layer_conf):
    layers = []
    for i in range(1, len(layer_conf)):
        np.random.seed(0)
        k = 1/math.sqrt(layer_conf[i]["hidden"])
        i_weight = np.random.rand(layer_conf[i-1]["units"], layer_conf[i]["hidden"]) * 2 * k - k

        h_weight = np.random.rand(layer_conf[i]["hidden"], layer_conf[i]["hidden"]) * 2 * k - k
        h_bias = np.random.rand(1, layer_conf[i]["hidden"]) * 2 * k - k

        o_weight = np.random.rand(layer_conf[i]["hidden"], layer_conf[i]["output"]) * 2 * k - k
        o_bias = np.random.rand(1, layer_conf[i]["output"]) * 2 * k - k

        layers.append(
            [i_weight, h_weight, h_bias, o_weight, o_bias]
        )
    return layers

In [ ]:
def forward(x, layers):
    hiddens = []
    outputs = []
    for i in range(len(layers)):
        i_weight, h_weight, h_bias, o_weight, o_bias = layers[i]
        hidden = np.zeros((x.shape[0], i_weight.shape[1]))
        output = np.zeros((x.shape[0], o_weight.shape[1]))
        for j in range(x.shape[0]):
            input_x = x[j,:][np.newaxis,:] @ i_weight
            hidden_x = input_x + hidden[max(j-1,0),:][np.newaxis,:] @ h_weight + h_bias
            # Activation.  tanh avoids outputs getting larger and larger.
            hidden_x = np.tanh(hidden_x)
            # Store hidden for use in backprop
            hidden[j,:] = hidden_x

            # Output layer
            output_x = hidden_x @ o_weight + o_bias
            output[j,:] = output_x
        hiddens.append(hidden)
        outputs.append(output)
    return hiddens, outputs[-1]

In [ ]:
def mse(actual, predicted):
    return np.mean((actual-predicted)**2)

def mse_grad(actual, predicted):
    return (predicted - actual)

In [ ]:
def backward(layers, x, lr, grad, hiddens):
    for i in range(len(layers)):
        i_weight, h_weight, h_bias, o_weight, o_bias = layers[i]
        hidden = hiddens[i]
        next_h_grad = None
        i_weight_grad, h_weight_grad, h_bias_grad, o_weight_grad, o_bias_grad = [0] * 5

        for j in range(x.shape[0] - 1, -1, -1):
            # 1,1
            # Add newaxis in the first dimension
            out_grad = grad[j,:][np.newaxis, :]

            # Output updates
            # 3x1 @ 1x1 = 3,1
            # np.newaxis creates a size 1 axis, in this case transposing matrix
            o_weight_grad += hidden[j,:][:, np.newaxis] @ out_grad
            # 1,1
            o_bias_grad += out_grad

            # Propagate gradient to hidden unit
            # 1,1 @ 1,3 = 1,3
            h_grad = out_grad @ o_weight.T

            if j < x.shape[0] - 1:
                # Then we multiply the gradient by the hidden weights to pull gradient from next hidden state to current hidden state
                hh_grad = next_h_grad @ h_weight.T
                # Add the gradients together to combine output contribution and hidden contribution
                h_grad += hh_grad

            # Pull the gradient across the current hidden nonlinearity
            # derivative of tanh is 1 - tanh(x) ** 2
            # So we take the output of tanh (next hidden state), and plug in
            tanh_deriv = 1 - hidden[j][np.newaxis,:] ** 2

            # next_h_grad @ np.diag(tanh_deriv_next) multiplies each element of next_h_grad by the deriv
            # Effect is to pull value across nonlinearity
            h_grad = np.multiply(h_grad, tanh_deriv)

            # Store to compute h grad for previous sequence position
            next_h_grad = h_grad.copy()

            # If we're not at the very beginning
            if j > 0:
                # 3,1 @ 1,3
                # Multiply input from previous layer by post-nonlinearity grad at current layer
                h_weight_grad += hidden[j-1][:, np.newaxis] @ h_grad
                h_bias_grad += h_grad

            # 3,1 (with newaxis) @ h_grad
            i_weight_grad += x[j,:][:,np.newaxis] @ h_grad

        lr = lr / x.shape[0]
        i_weight -= i_weight_grad * lr
        h_weight -= h_weight_grad * lr
        h_bias -= h_bias_grad * lr
        o_weight -= o_weight_grad * lr
        o_bias -= o_bias_grad * lr
        layers[i] = [i_weight, h_weight, h_bias, o_weight, o_bias]
    return layers

In [ ]:
epochs = 250
lr = 1e-5

layer_conf = [
    {"type":"input", "units": 3},
    {"type": "rnn", "hidden": 4, "output": 1}
]
layers = init_params(layer_conf)

for epoch in range(epochs):
    sequence_len = 7
    epoch_loss = 0
    for j in range(train_x.shape[0] - sequence_len):
        seq_x = train_x[j:(j+sequence_len),]
        seq_y = train_y[j:(j+sequence_len),]
        hiddens, outputs = forward(seq_x, layers)
        grad = mse_grad(seq_y, outputs)
        params = backward(layers, seq_x, lr, grad, hiddens)
        epoch_loss += mse(seq_y, outputs)

    if epoch % 10 == 0:
        sequence_len = 7
        valid_loss = 0
        for j in range(valid_x.shape[0] - sequence_len):
            seq_x = valid_x[j:(j+sequence_len),]
            seq_y = valid_y[j:(j+sequence_len),]
            _, outputs = forward(seq_x, layers)
            valid_loss += mse(seq_y, outputs)

        print(f"Epoch: {epoch} train loss {epoch_loss / len(train_x)} valid loss {valid_loss / len(valid_x)}")

In [ ]:
import numpy as np

class LSTM:
    def __init__(self, input_dim, hidden_dim, output_dim):
        # Dimensions for input layer, hidden layer and output layer
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        # Initialize weights and biases for gates
        self.init_weights()

    def init_weights(self):
        # Can be upgraded to Glorot (Xavier) initialisation or He initialisation
        self.Wf = np.random.randn(self.hidden_dim, self.input_dim + self.hidden_dim)
        self.Wi = np.random.randn(self.hidden_dim, self.input_dim + self.hidden_dim)
        self.Wc = np.random.randn(self.hidden_dim, self.input_dim + self.hidden_dim)
        self.Wo = np.random.randn(self.hidden_dim, self.input_dim + self.hidden_dim)

        self.bf = np.zeros((self.hidden_dim, 1))
        self.bi = np.zeros((self.hidden_dim, 1))
        self.bc = np.zeros((self.hidden_dim, 1))
        self.bo = np.zeros((self.hidden_dim, 1))

    # Activation functions and their derivatives
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def tanh(self, x):
        return np.tanh(x)

    def sigmoid_derivative(self, sigmoid_output):
        return sigmoid_output * (1 - sigmoid_output)

    def tanh_derivative(self, tanh_output):
        return 1 - np.power(tanh_output, 2)

    # Forward propagation
    def forward(self, x):
        """
        x: Input data, shape (batch_size, sequence_length, input_dim)
        """
        batch_size, sequence_length, _ = x.shape
        h_prev = np.zeros((batch_size, self.hidden_dim))
        c_prev = np.zeros((batch_size, self.hidden_dim))
        all_hiddens = np.zeros((batch_size, sequence_length, self.hidden_dim))
        all_outputs = np.zeros((batch_size, sequence_length, self.output_dim))

        for t in range(sequence_length):
            xt = x[:, t, :]
            combined = np.hstack((h_prev, xt))  # Combines the hidden layer with the input lair
            ft = self.sigmoid(np.dot(combined, self.Wf.T) + self.bf.T)  # Forget gate
            it = self.sigmoid(np.dot(combined, self.Wi.T) + self.bi.T)  # Input gate
            ct_ = np.tanh(np.dot(combined, self.Wc.T) + self.bc.T)     # Cell candidate
            ot = self.sigmoid(np.dot(combined, self.Wo.T) + self.bo.T) # Output gate

            c_curr = ft * c_prev + it * ct_  # Update cell's state
            h_curr = ot * np.tanh(c_curr)   # Update hidden state

            # Update states for the next step
            h_prev = h_curr
            c_prev = c_curr

            # memorize the hidden state for the whole sequence
            all_hiddens[:, t, :] = h_curr

            # We can add an output state if necessary
            # output = some_activation(np.dot(h_curr, W_output) + b_output)
            # all_outputs[:, t, :] = output

        return all_hiddens, all_outputs



    def step(self, x_t, h_prev, c_prev):
        # Combine hidden and input state for matrix calculus
        combined = np.hstack([h_prev, x_t])

        # Gates
        ft = self.sigmoid(np.dot(self.Wf, combined) + self.bf)  # Forget gate
        it = self.sigmoid(np.dot(self.Wi, combined) + self.bi)  # Input gate
        ct_ = self.tanh(np.dot(self.Wc, combined) + self.bc)    # Cell's candidate(cell gate)
        ot = self.sigmoid(np.dot(self.Wo, combined) + self.bo)  # Output gate

        # Update states for the next step
        c_curr = ft * c_prev + it * ct_
        h_curr = ot * self.tanh(c_curr)

        return h_curr, c_curr

    def backward(self, x, h, c, y, targets, dh_next, dc_next):
        """
        x: input data at all time steps (batch_size, sequence_length, input_dim)
        h: hidden states at all time steps (batch_size, sequence_length, hidden_dim)
        c: cell states at all time steps (batch_size, sequence_length, hidden_dim)
        y: outputs at all time steps (batch_size, sequence_length, output_dim)
        targets: ground truth for the outputs
        dh_next: gradient of the loss w.r.t. next hidden state
        dc_next: gradient of the loss w.r.t. next cell state
        """
        batch_size, sequence_length, _ = x.shape

        # Gradient accumulators
        dWf, dWi, dWc, dWo = [np.zeros_like(W) for W in (self.Wf, self.Wi, self.Wc, self.Wo)]
        dbf, dbi, dbc, dbo = [np.zeros_like(b) for b in (self.bf, self.bi, self.bc, self.bo)]

        # Iterate over the sequence backwards
        for t in reversed(range(sequence_length)):
            # Compute gradients of the output layer
            dy = (y[:, t, :] - targets[:, t, :])  # shape: (batch_size, output_dim)
            dWhy = np.dot(dy.T, h[:, t, :])
            dby = np.sum(dy, axis=0, keepdims=True).T

            # Gradients of the gates using the final step gradients and previous gradients
            do = dh_next * np.tanh(c[:, t, :])
            dc = (dc_next + dh_next * self.sigmoid(self.Wo) * self.tanh_derivative(np.tanh(c[:, t, :])))
            dc_f = dc * self.sigmoid(self.Wf)
            di = dc * self.sigmoid(self.Wi)
            dg = dc * self.tanh(c[:, t, :])

            # Calculate gate derivatives
            dfo = do * self.sigmoid_derivative(self.sigmoid(self.Wo))
            dfi = di * self.sigmoid_derivative(self.sigmoid(self.Wi))
            dfc = dg * self.tanh_derivative(self.sigmoid(self.Wc))
            dfo_next = self.sigmoid_derivative(np.dot(x[:, t, :], self.Wo.T) + self.bo)
            dfi_next = self.sigmoid_derivative(np.dot(x[:, t, :], self.Wi.T) + self.bi)
            dfc_next = self.tanh_derivative(np.dot(x[:, t, :], self.Wc.T) + self.bc)

            # Accumulate gradients
            dWf += np.dot(dfo.T, np.hstack([h[:, t-1, :], x[:, t, :]]))
            dWi += np.dot(dfi.T, np.hstack([h[:, t-1, :], x[:, t, :]]))
            dWc += np.dot(dfc.T, np.hstack([h[:, t-1, :], x[:, t, :]]))
            dWo += np.dot(dfo_next.T, np.hstack([h[:, t-1, :], x[:, t, :]]))
            dbf += np.sum(dfo, axis=0)
            dbi += np.sum(dfi, axis=0)
            dbc += np.sum(dfc, axis=0)
            dbo += np.sum(dfo_next, axis=0)

            # Prepare for next iteration
            dh_next = np.dot(dfo.T, self.Wf) + np.dot(dfi.T, self.Wi) + np.dot(dfc.T, self.Wc) + np.dot(dfo_next.T, self.Wo)
            dc_next = dc_f

        # Update weights and biases
        self.Wf -= self.learning_rate * dWf
        self.Wi -= self.learning_rate * dWi
        self.Wc -= self.learning_rate * dWc
        self.Wo -= self.learning_rate * dWo
        self.bf -= self.learning_rate * dbf
        self.bi -= self.learning_rate * dbi
        self.bc -= self.learning_rate * dbc
        self.bo -= self.learning_rate * dbo

        return dWhy, dby

    def update_weights(self, gradients):
        # Extract each gradient
        dWf, dWi, dWc, dWo, dbf, dbi, dbc, dbo = gradients

        # Update weights and biases
        self.Wf -= self.learning_rate * dWf
        self.Wi -= self.learning_rate * dWi
        self.Wc -= self.learning_rate * dWc
        self.Wo -= self.learning_rate * dWo

        self.bf -= self.learning_rate * dbf
        self.bi -= self.learning_rate * dbi
        self.bc -= self.learning_rate * dbc
        self.bo -= self.learning_rate * dbo

# lstm = LSTM(input_dim=10, hidden_dim=50, output_dim=1)


In [ ]:
def train_lstm(model, data, epochs, learning_rate):
    input_data, target_data = data
    loss_history = []

    for epoch in range(epochs):
        total_loss = 0
        for i in range(len(input_data)):
            # Forward propagation
            hiddens, outputs = model.forward(input_data[i])

            # Loss calculus (MSE - can be any other function)
            loss = np.mean((outputs - target_data[i]) ** 2)
            total_loss += loss

            # Backpropagation
            gradients = model.backward(input_data[i], hiddens, outputs, target_data[i])

            # Update weights
            model.update_weights(gradients, learning_rate)

        # Mean loss per epoch
        average_loss = total_loss / len(input_data)
        loss_history.append(average_loss)
        print(f'Epoch: {epoch + 1}/{epochs}, Batch: {i + 1}/{len(input_data)}, Loss: {average_loss}')

    return loss_history

In [ ]:
# To save
import json

def save_model(self, filename):
    model_params = {
        'Wf': self.Wf.tolist(), 'Wi': self.Wi.tolist(),
        'Wc': self.Wc.tolist(), 'Wo': self.Wo.tolist(),
        'bf': self.bf.tolist(), 'bi': self.bi.tolist(),
        'bc': self.bc.tolist(), 'bo': self.bo.tolist()
    }
    with open(filename, 'w') as f:
        json.dump(model_params, f)

def load_model(self, filename):
    with open(filename, 'r') as f:
        model_params = json.load(f)
    self.Wf = np.array(model_params['Wf'])
    self.Wi = np.array(model_params['Wi'])
    self.Wc = np.array(model_params['Wc'])
    self.Wo = np.array(model_params['Wo'])
    self.bf = np.array(model_params['bf'])
    self.bi = np.array(model_params['bi'])
    self.bc = np.array(model_params['bc'])
    self.bo = np.array(model_params['bo'])


In [ ]:
I have a stomach ache.

I - 0
have - 0
a - 0
stomach - B-SYM
ache - I-SYM